# [TR-003] Chew-Mandelstam

In [ ]:
%%sh
pip install ampform==0.10.1 matplotlib==3.4.2 mpl-interactions==0.17.12 numpy==1.21.0 qrules==0.9.0 sympy==1.8 > /dev/null

This report is an attempt to visualize the Chew-Mandelstam function described in {pdg-review}`2020; Resonances; pp.9-10` (Section 49.3.5).

In [ ]:
%matplotlib widget
import os
import warnings

import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt
import numpy as np
import qrules
import symplot
import sympy as sp
from ampform.dynamics import (
    ComplexSqrt,
    breakup_momentum_squared,
    phase_space_factor_complex,
)
from IPython.display import Math

warnings.filterwarnings("ignore")
STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)
PDG = qrules.load_pdg()

## S-wave

As can be seen in Eq. (49.35), the Chew-Mandelstam function $\Sigma_a$ for a particle $a$ decaying to particles $1, 2$ has a simple form for angular momentum $L=0$ ($S$-wave). The only question is how to deal with negative values for the squared break-up momentum $q_a^2$. Here, we will use AmpForm's {class}`~ampform.dynamics.math.ComplexSqrt`:

In [ ]:
q_squared_symbol = sp.Symbol("q_a^{2}", real=True)
q_a_expr = ComplexSqrt(q_squared_symbol)
Math(f"q_a = {sp.latex(q_a_expr)} = {sp.latex(q_a_expr.evaluate())}")

In [ ]:
def breakup_momentum(s, m1, m2):
    q_squared = breakup_momentum_squared(s, m1, m2)
    return ComplexSqrt(q_squared)

In [ ]:
phsp_scale_factor = 16 * sp.pi ** 2


def chew_mandelstam_s_wave(s, m1, m2):
    q = breakup_momentum(s, m1, m2)
    left_term = sp.Mul(
        2 * q / sp.sqrt(s),
        sp.log((m1 ** 2 + m2 ** 2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2)),
        evaluate=False,
    )
    right_term = m1 ** 2 - m2 ** 2
    right_term *= 1 / s - 1 / (m1 + m2) ** 2
    right_term *= sp.log(m1 / m2)
    return (left_term - right_term) / phsp_scale_factor

To check whether this implementation is correct, let's plug some {class}`~sympy.core.symbol.Symbol`s into this function and compare it to Eq. (49.35) on {pdg-review}`2020; Resonances; p.9`:

In [ ]:
s, m1, m2 = sp.symbols("s m1 m2", real=True)
chew_mandelstam_s_wave_expr = chew_mandelstam_s_wave(s, m1, m2)

In [ ]:
chew_mandelstam_s_wave_symbolic = chew_mandelstam_s_wave_expr.subs(
    2 * breakup_momentum(s, m1, m2),
    2 * sp.Symbol("q_a"),
)
chew_mandelstam_s_wave_symbolic *= phsp_scale_factor
sp.Mul(1 / phsp_scale_factor, chew_mandelstam_s_wave_symbolic, evaluate=False)

It should be noted that this equation is not well-defined along the real axis, that is, for $\mathrm{Im}(s) = 0$. For this reason, we split $s$ into a real part $s'$ with a small imaginary offset (the PDG indicates this with $s+0i$). We parametrized this imaginary offset with $\epsilon$, and for the interactive plot, we do so with a power of $10$:

In [ ]:
epsilon = sp.Symbol("epsilon", real=True, positive=True)
s_prime = sp.Symbol(R"s^{\prime}", real=True)
s_plus = s_prime + sp.I * sp.Pow(10, -epsilon)

In [ ]:
Math(fR"{sp.latex(s)} \to {sp.latex(s_plus)}")

We are now ready to use [`mpl_interactions`](https://mpl-interactions.rtfd.io) and AmpForm's {mod}`symplot` to visualize this function:

In [ ]:
chew_mandelstam_s_wave_prime = chew_mandelstam_s_wave_expr.subs(s, s_plus)
np_chew_mandelstam_s_wave, sliders = symplot.prepare_sliders(
    expression=chew_mandelstam_s_wave_prime,
    plot_symbol=s_prime,
)
np_phase_space_factor = sp.lambdify(
    args=(s_prime, m1, m2, epsilon),
    expr=phase_space_factor_complex(s_plus, m1, m2),
    modules="numpy",
)

As starting values for the interactive plot, we assume $\pi\eta$ scattering (just like in the PDG section) and use their masses as values for $m_1$ and $m_1$, respectively.

In [ ]:
s_min, s_max = -0.15, 1.4
m1_val = PDG["pi0"].mass
m2_val = PDG["eta"].mass

plot_domain = np.linspace(s_min, s_max, 500)
sliders.set_ranges(
    m1=(0, 2, 200),
    m2=(0, 2, 200),
    epsilon=(1, 12),
)
sliders.set_values(
    m1=m1_val,
    m2=m2_val,
    epsilon=4,
)

For comparison, we plot the Chew-Mandelstam function for $S$-waves next to AmpForm's {func}`~ampform.dynamics.phase_space_factor_complex`. Have a look at the resulting plots and compare to Figure 49.3 on {pdg-review}`2020; Resonances; p.10`.

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(11, 4.5), tight_layout=True)
ax1, ax2 = axes
for ax in axes:
    ax.axhline(0, linewidth=0.5, c="black")

ylim = (-1, +1)
y_factor = 16 * np.pi
controls = iplt.axvline(
    lambda *args, **kwargs: (kwargs["m1"] + kwargs["m2"]) ** 2,
    **sliders,
    c="grey",
    linewidth=0.5,
    label=R"$s_\mathrm{thr}$",
    ax=ax1,
)
iplt.axvline(
    lambda *args, **kwargs: (kwargs["m1"] + kwargs["m2"]) ** 2,
    controls=controls,
    c="grey",
    linewidth=0.5,
    label=R"$s_\mathrm{thr}$",
    ax=ax2,
)
iplt.plot(
    plot_domain,
    lambda *args, **kwargs: (
        y_factor * 1j * np_phase_space_factor(*args, **kwargs)
    ).real,
    label="Real part",
    controls=controls,
    ylim=ylim,
    alpha=0.7,
    ax=ax1,
    c="black",
    linestyle="dashed",
)
iplt.plot(
    plot_domain,
    lambda *args, **kwargs: (
        y_factor * 1j * np_phase_space_factor(*args, **kwargs)
    ).imag,
    label="Imag part",
    controls=controls,
    ylim=ylim,
    alpha=0.7,
    ax=ax1,
    c="red",
)

iplt.plot(
    plot_domain,
    lambda *args, **kwargs: y_factor
    * np_chew_mandelstam_s_wave(*args, **kwargs).real,
    label="Real part",
    controls=controls,
    ylim=ylim,
    alpha=0.7,
    ax=ax2,
    c="black",
    linestyle="dashed",
)
iplt.plot(
    plot_domain,
    lambda *args, **kwargs: y_factor
    * np_chew_mandelstam_s_wave(*args, **kwargs).imag,
    label="Imag part",
    controls=controls,
    ylim=ylim,
    alpha=0.7,
    ax=ax2,
    c="red",
)

for ax in axes:
    ax.legend(loc="lower right")
    ax.set_xticks(np.arange(0, 1.21, 0.3))
    ax.set_yticks(np.arange(-1, 1.1, 0.5))
    ax.set_xlabel("$s$ (GeV$^2$)")

ax1.set_ylabel(R"$16\pi \; i\rho(s)$")
ax2.set_ylabel(R"$16\pi \; \Sigma(s)$")
ax1.set_title(R"Complex phase space factor $\rho$")
ax2.set_title("Chew-Mandelstam $S$-wave ($L=0$)")
plt.show()

In [ ]:
if STATIC_WEB_PAGE:
    from IPython.display import SVG

    output_file = "chew-mandelstam-s-wave.svg"
    plt.savefig(output_file)
    display(SVG(output_file))